In [42]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense
print(tf.__version__)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder #Used for converting labels to numbers(0 or 1)
dataset = pd.read_csv('CustomerModelling.csv')
dataset.head()
x = dataset.drop(labels=['RowNumber','CustomerId','Surname','Exited'], axis=1)
y = dataset['Exited']
x.head()
#y.head()

2.1.0


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [43]:
label1 = LabelEncoder()
x['Geography'] = label1.fit_transform(x['Geography'])
x.head(20)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,Female,42,2,0.00,1,1,1,101348.88
1,608,2,Female,41,1,83807.86,1,0,1,112542.58
2,502,0,Female,42,8,159660.80,3,1,0,113931.57
3,699,0,Female,39,1,0.00,2,0,0,93826.63
4,850,2,Female,43,2,125510.82,1,1,1,79084.10
5,645,2,Male,44,8,113755.78,2,1,0,149756.71
6,822,0,Male,50,7,0.00,2,1,1,10062.80
7,376,1,Female,29,4,115046.74,4,1,0,119346.88
8,501,0,Male,44,4,142051.07,2,0,1,74940.50
9,684,0,Male,27,2,134603.88,1,1,1,71725.73


In [44]:
label2 = LabelEncoder()
x['Gender'] = label2.fit_transform(x['Gender'])
x.head(10)
#Since Gender and Geography are not have any waightage based on the number ,these are catagorical valyes .(0.1,2,3..all ahave same).
#We need to convert into one hot encoding
x = pd.get_dummies(x,drop_first=True,columns=['Geography'])

In [40]:
x.head(10)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_1,Geography_2
0,619,0,42,2,0.00,1,1,1,101348.88,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,1
5,645,1,44,8,113755.78,2,1,0,149756.71,0,1
6,822,1,50,7,0.00,2,1,1,10062.80,0,0
7,376,0,29,4,115046.74,4,1,0,119346.88,1,0
8,501,1,44,4,142051.07,2,0,1,74940.50,0,0
9,684,1,27,2,134603.88,1,1,1,71725.73,0,0


In [61]:
###future standardization, since some of the values are large numbers we need to scale
from sklearn.preprocessing import StandardScaler
x_train , x_test ,y_train, y_test = train_test_split(x,y,test_size=0.2,random_state = 0,stratify = y)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)
y_train

493     1
6839    0
170     0
4958    0
4271    0
       ..
6727    1
4674    0
6399    1
872     0
8389    0
Name: Exited, Length: 8000, dtype: int64

In [66]:
model = Sequential()
model.add(Dense(x.shape[1],activation='relu',input_dim=x.shape[1])) #First input layer
model.add(Dense(128,activation='relu')) #Hidden layer with 128 newrons and no input_dim required since it is not a inputlayer
model.add(Dense(1,activation='sigmoid')) #Outplut layer , here ther is 2 only 2 values (1 or 0) hence we required only single node ,1 means customer with tge bank and o means customer going to leave the bank.
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy']) #adam optimizer is the stocastic gradiant optimizer , we are not used batch radiant gradiant descent alagorithm which take a lot of time to train the model
#loss function we have  binary_crossentropy  , since we have to predict binary classificatin other wise we have to use other

In [67]:
#Start fitting the model
model.fit(x_train,y_train.to_numpy(),batch_size=10,epochs = 10 , verbose = 1) 
#since x_train is numpy array , the y_train alos should be numpy array , batchsize means in stocastic gradiants how many rows or datapoints 
#going to take caliculate the new weights . If batch size=1 that means every input it will update the weights that becomes toomuch random ,
#and if batchsize = total size of data ,then the stocastica gradiant becomes a kind of batch gradiant .
#epochs = 10   we can try for 20 also , we need to see the acurracy , if the accurracy not much changed we should not increase the epoch
#verbose means verbose mode give some logs and give progress bar

Train on 8000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 156us/sample - loss: 0.4497 - accuracy: 0.8101
Epoch 2/10
8000/8000 [==============================] - 1s 103us/sample - loss: 0.3683 - accuracy: 0.8484
Epoch 3/10
8000/8000 [==============================] - 1s 114us/sample - loss: 0.3500 - accuracy: 0.8565
Epoch 4/10
8000/8000 [==============================] - 1s 110us/sample - loss: 0.3442 - accuracy: 0.8599
Epoch 5/10
8000/8000 [==============================] - 1s 107us/sample - loss: 0.3395 - accuracy: 0.8620
Epoch 6/10
8000/8000 [==============================] - 1s 106us/sample - loss: 0.3364 - accuracy: 0.8643
Epoch 7/10
8000/8000 [==============================] - 1s 106us/sample - loss: 0.3337 - accuracy: 0.8651
Epoch 8/10
8000/8000 [==============================] - 1s 104us/sample - loss: 0.3324 - accuracy: 0.8645
Epoch 9/10
8000/8000 [==============================] - 1s 105us/sample - loss: 0.3304 - accuracy: 0.8694
Epoch 10/10
8000/8000 [=

In [68]:
y_pred = model.predict_classes(x_test)

In [70]:
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [71]:
y_test

1344    1
8167    0
4747    0
5004    1
3124    1
       ..
9107    0
8249    0
8337    0
6279    1
412     0
Name: Exited, Length: 2000, dtype: int64

In [73]:
model.evaluate(x_test,y_test.to_numpy()) #evoluating the accuracy using tensorflow

2000/2000 [==============================] - 0s 64us/sample - loss: 0.3448 - accuracy: 0.8615


[0.3447940695285797, 0.8615]

In [82]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test,y_pred)

array([[1515,   78],
       [ 199,  208]], dtype=int64)

In [84]:
accuracy_score(y_test,y_pred)   #Accurracy is same bwter tensorflow methos and sklearn

0.8615